In [ ]:
!pip install numpy==1.23.1

In [ ]:
!pip install mxnet
!pip install gluonnlp
!pip install sentencepiece
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 7.9 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661658 sha256=45e22004e7eeae8de412465b4666a8237b308a37c3320193a7a2f96070497c4a
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-p

In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-pzewfmnq/kobert-tokenizer_1388189999ab47f0b21bfb49f612a779
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-pzewfmnq/kobert-tokenizer_1388189999ab47f0b21bfb49f612a779
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=c13e85dfddae357014e50ccf9ed11fdffdc44b11cf39bf6b70f408258a690bb5
  Stored in directory: /tmp/pip-ephem-wheel-cache-mebq4r6c/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
# Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
device = torch.device("cuda:0")

In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [ ]:
#구글 드라이브 import
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):

        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))


    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 128
warmup_ratio = 0.1
max_grad_norm = 1
# log_interval = 200

In [38]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/유런 24 여름 방학 프로젝트/Test/test_preprocessing_final.csv")

#feature label 나누기
data_list = []
for title, label in zip(df['processed_title'], df['label']):
  data = []
  data.append(title)
  data.append(str(label))

  data_list.append(data)

In [39]:
df_test = data_list

In [40]:
df_test = [[str(text), int(label)] for text, label in df_test]

In [41]:
tok=tokenizer.tokenize

data_test = BERTDataset(df_test,0, 1, tok, vocab,  max_len, True, False)

In [42]:
from torch.utils.data import SequentialSampler

# test 데이터 로더
test_dataloader = DataLoader(
            data_test,
            sampler = SequentialSampler(data_test), # Pull out batches sequentially.
            batch_size = 64
            )

In [43]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [44]:
#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc


In [45]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [51]:
# 평가 함수
def evaluate_model_on_test_data(model, test_dataloader, device=torch.device("cuda:0")):
    model.to(device)
    model.eval()

    loss_fn = nn.CrossEntropyLoss()

    print("Running Test Evaluation...")

    t0 = time.time()

    # 변수 초기화
    total_test_accuracy = 0
    total_test_loss = 0

    # 테스트 데이터셋에 대한 평가
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)

        with torch.no_grad():
            out = model(token_ids, valid_length, segment_ids)
            loss = loss_fn(out, label)

        # loss 와 accuracy 누적
        total_test_loss += loss.item()
        total_test_accuracy += calc_accuracy(out, label)

    # average loss와  average accuracy 계산
    avg_test_accuracy = total_test_accuracy / len(test_dataloader)
    avg_test_loss = total_test_loss / len(test_dataloader)

    # 평가 시간 측정
    test_time = format_time(time.time() - t0)

    # 결과 출력
    print("  Test Accuracy: {0:.4f}".format(avg_test_accuracy))
    print("  Test Loss: {0:.4f}".format(avg_test_loss))
    print("  Test Evaluation took: {:}".format(test_time))

    # 결과 반환
    return {
        'Test Loss': avg_test_loss,
        'Test Accuracy': avg_test_accuracy,
        'Test Time': test_time
    }



## 함수 적용

In [48]:
model_plain = BERTClassifier(bertmodel, dr_rate  = 0.5)

In [49]:
model_plain.load_state_dict(torch.load('/content/drive/MyDrive/유런 24 여름 방학 프로젝트/BERT/KoBERT/kobert_plain.pth'))

<All keys matched successfully>

In [52]:
evaluate_model_on_test_data(model_plain, test_dataloader, device=torch.device("cuda:0"))

Running Test Evaluation...
  Test Accuracy: 0.8976
  Test Loss: 0.4420
  Test Evaluation took: 0:00:05


{'Test Loss': 0.44201750804980594,
 'Test Accuracy': 0.8976102941176471,
 'Test Time': '0:00:05'}

In [53]:
model_llrd = BERTClassifier(bertmodel, dr_rate  = 0.5)

In [54]:
model_llrd.load_state_dict(torch.load('/content/drive/MyDrive/유런 24 여름 방학 프로젝트/BERT/KoBERT/kobert_llrd.pth'))

<All keys matched successfully>

In [55]:
evaluate_model_on_test_data(model_llrd, test_dataloader, device=torch.device("cuda:0"))

Running Test Evaluation...
  Test Accuracy: 0.8982
  Test Loss: 0.5367
  Test Evaluation took: 0:00:06


{'Test Loss': 0.5367137044668198,
 'Test Accuracy': 0.8981617647058824,
 'Test Time': '0:00:06'}